### 1. Import dependencies

In [ ]:
!pip install 'stable-baselines3[extra]'

In [1]:
import os 
import gym 
from stable_baselines3 import PPO 
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

### 2. Load Environment 

In [5]:
environmnet_name = 'CartPole-v0'
env = gym.make(environmnet_name)

In [3]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset() # initialize environment state - returns initial set of observations from the environment
    done = False 
    score = 0

    while not done:
        env.render() # visualize environment 
        action = env.action_space.sample() # generate randome action. In this case action_space is Discrete(2) -> possible actions 0 or 1 
        n_state, reward, done, info = env.step(action) # apply action to environment - returns next set of observations, reward, if episode is done and info
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    
env.close()

2022-09-11 08:54:53.899 Python[2727:38121] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/7b/p3zx9c7x72sgh_b4k3l6sdp40000gn/T/org.python.python.savedState


Episode:1 Score:12.0
Episode:2 Score:25.0
Episode:3 Score:12.0
Episode:4 Score:21.0
Episode:5 Score:13.0


### Understanding our Environment

In [4]:
env.action_space # our action space only has two available actions

Discrete(2)

In [5]:
env.action_space.sample() # 0 - push cart to left, 1 - push cart to right 

1

In [6]:
env.observation_space 

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [7]:
env.observation_space.sample() # cart position, cart velocity, pole angle, pole angular velocity 

array([-4.2437119e+00, -7.3994642e+37, -3.2807064e-01,  1.1330229e+38],
      dtype=float32)

### 3. Training an RL Model 

In [8]:
log_path = os.path.join('Training', 'Logs')

In [4]:
env = gym.make(environmnet_name)
env = DummyVecEnv([lambda:env]) # wrapping our environement in dummy vectorized environment 
model = PPO('MlpPolicy', env, verbose=1,tensorboard_log=log_path) # think of this as our agent 
# verbose = 1 says that we want to log our output

NameError: name 'environmnet_name' is not defined

In [10]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_6
-----------------------------
| time/              |      |
|    fps             | 6191 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 3798        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008948295 |
|    clip_fraction        | 0.0687      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.004      |
|    learning_rate        | 0.0003      |
|    loss                 | 6.07        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0107     |
|    value_loss           | 48.2        |
-----------------------------------------
---

### 4. Save and Reload Model

In [11]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [13]:
model.save(PPO_Path)

In [14]:
del model # simulating "deployment" by deleting model from memory

In [15]:
model = PPO.load(PPO_Path, env=env) # then reloading 

### 5. Evaluation 

In [16]:
evaluate_policy(model, env, n_eval_episodes=10, render=True) # the output is average reward per episode and standard deviation 

/opt/homebrew/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [17]:
env.close()

### 6. Test Model

In [19]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset() # initialize environment state - returns initial set of observations from the environment
    done = False 
    score = 0

    while not done:
        env.render() # visualize environment 
        action, _states = model.predict(obs)
        obs, reward, done, info = env.step(action) # apply action to environment - returns next set of observations, REWARD, if episode is done and info
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    
env.close()

Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[200.]
Episode:5 Score:[200.]


### 7. Viewing Logs in Tensorboard

In [20]:
training_log_path = os.path.join(log_path, 'PPO_2')
save

In [23]:
!tensorboard --logdir={training_log_path} ## Don't actually do this - run in terminal as this will run indefinitely and lockup notebook 

TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


### 8. Adding a Callback to the Training Stage 

In [2]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [3]:
save_path = os.path.join('Training', 'Saved Models')

In [6]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
# eval_callback is going to be triggered after each training run 
eval_callback = EvalCallback(env,
                            callback_on_new_best=stop_callback, # call the stop callback each time there is a new best model 
                            eval_freq=10000,
                            best_model_save_path=save_path,
                            verbose=1)

In [9]:
model = PPO('MlpPolicy',env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [20]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_7
-----------------------------
| time/              |      |
|    fps             | 6387 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 3929        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008531956 |
|    clip_fraction        | 0.0862      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00612    |
|    learning_rate        | 0.0003      |
|    loss                 | 4.88        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0141     |
|    value_loss           | 49          |
-----------------------------------------
---

/opt/homebrew/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


### 9. Changing Policies 

In [11]:
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])] # pi - custom actor ,vf - value function 

In [12]:
model = PPO('MlpPolicy',env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [14]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_8
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.1     |
|    ep_rew_mean     | 21.1     |
| time/              |          |
|    fps             | 5960     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 30.1       |
|    ep_rew_mean          | 30.1       |
| time/                   |            |
|    fps                  | 3026       |
|    iterations           | 2          |
|    time_elapsed         | 1          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01582878 |
|    clip_fraction        | 0.205      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.681     |
|    explained_variance   | -0.00282   |
|    learning_

/opt/homebrew/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


### 10.Alternate Algorithm 

In [16]:
from stable_baselines3 import DQN

In [18]:
model = DQN('MlpPolicy',env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [19]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 20.2     |
|    ep_rew_mean      | 20.2     |
|    exploration_rate | 0.962    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 12231    |
|    time_elapsed     | 0        |
|    total_timesteps  | 81       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 19.8     |
|    ep_rew_mean      | 19.8     |
|    exploration_rate | 0.925    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 14984    |
|    time_elapsed     | 0        |
|    total_timesteps  | 158      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 17.4     |
|    ep_rew_mean      | 17.4     |
|    exploration_rate | 0.901    |
| time/               | 